# Get Process ID from a Kernel Module

In a previous *Hello World* about Ftrace, [Debug the Linux Kernel Using Ftrace](../ftrace/ftrace.ipynb#ring_buff_overflow), I found that the functions in a kernel module are not always running within a specific process context, here are the scripts:
```sh
echo 'function' > $DEBUGFS/tracing/current_tracer

insmod hello_ftrace/hello_ftrace.ko    # install module
rmmod hello_ftrace       # remove module
filter_trace 'Ftrace'

echo 'nop' > $DEBUGFS/tracing/current_tracer
```
Here's the output from */sys/kernel/debug/tracing/trace*:
```txt
# tracer: function
#
# entries-in-buffer/entries-written: 2/2   #P:8
#
#                        _-----=> irqs-off/BH-disabled
#                       / _----=> need-resched
#                      | / _---=> hardirq/softirq
#                      || / _--=> preempt-depth
#                      ||| / _-=> migrate-disable
#                      |||| /     delay
#   TASK-PID     CPU#  |||||  TIMESTAMP  FUNCTION
#      | |         |   |||||     |         |
  insmod-10199   [004] ..... 10808.424001: 0xffffffffc2071005: Hello, Ftrace!
   rmmod-10201   [000] ..... 10808.482073: 0xffffffffc2071045: Good bye, Ftrace!
```

A module in kernel, like *helloworld.ko*, usually serves like a shared object. When a process A, no matter it's a user space process or a kernel space thread, calls a functoin `func()` from the 'helloworld.ko', the kernel will run the function `func()` on behalf of process A. If the `func()` wants to get the current process ID, the ID should be the same as of process A.

Let's build a simple kernel module *hello_kernel.ko* to define a function `read_mypid()`. The only thing that `read_mypid()` function performs, is to output the current process ID to a `sysfs` file, */sys/module/&lt;module&gt;/parameters/mypid*. According to the previous deduction, whoever reads *mypid*, the file will return its own PID. For example, command `cat mypid` will output the PID of `cat` command, command `tail mypid` will output the PID of `tail` command. OK, let's try it:

```C
static int read_mypid(char* buff, const struct kernel_param* kp)
{
    return sprintf(buff, "%d", current->pid);
}

static const struct kernel_param_ops mypid_ops = {
    .get = read_mypid
};

module_param_cb(mypid, &mypid_ops, NULL, 0444);
MODULE_PARM_DESC(mypid, "get my process id");
```
The source tree is show below:  
hello_kernel/  
├── [hello_initexit.c](hello_kernel/hello_initexit.c)  
├── [hello_module_param.c](hello_kernel/hello_module_param.c)  
└── [Makefile](hello_kernel/Makefile)

In [3]:
# build the .ko
make -C hello_kernel | grep '.ko$'

  LD [M]  /home/clymber/GitHub/kerdevel/module_getpid/hello_kernel/hello_getpid.ko


In [4]:
# install .ko
sudo insmod hello_kernel/hello_getpid.ko
ls -l /sys/module/hello_getpid/parameters/mypid

-r--r--r-- 1 root root 4096 Jun  2 20:16 /sys/module/hello_getpid/parameters/mypid


If we `cat` the file *mypid*, it should output the PID of `cat`:

In [5]:
cat /sys/module/hello_getpid/parameters/mypid

11915


If we `tail` the file *mypid*, it should output the PID of `tail`:

In [6]:
tail /sys/module/hello_getpid/parameters/mypid

11916


But how do we know the pid of `cat` or `tail`? The command `ps` is a good choice, but I have a simpler one:

In [7]:
bash -c '
printf "my current pid is: $$\n"
printf "pid got from kernel is: "

# we run `cat` with `exec` to let `cat` to have the same pid
exec cat /sys/module/hello_getpid/parameters/mypid
'

my current pid is: 11917
pid got from kernel is: 11917


So, the conclusion is simple: a kernel module does not have a PID. It's a shared object in the kernel space that serves other processes.